In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 3, Finished, Available, Finished)

In [3]:
%%sql
create TABLE if not EXISTS Ecommere_Lh.gold_shipmode
(
    ShipMode_ID Long    ,
    Ship_Mode string    ,
    Created_TS timestamp    ,
    Modified_TS timestamp   
)

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
max_date=spark.sql("SELECT coalesce(max(Modified_TS),'1900-01-01') FROM Ecommere_Lh.gold_shipmode").first()[0]

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 7, Finished, Available, Finished)

In [6]:
max_date

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 8, Finished, Available, Finished)

'1900-01-01'

In [15]:
df_bronze = spark.sql("SELECT distinct Ship_Mode from Ecommere_Lh.bronze_sales where Modified_TS > '{}'".format(max_date))

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 17, Finished, Available, Finished)

In [16]:
max_id =spark.sql("select coalesce(max(ShipMode_ID),0) from Ecommere_Lh.gold_shipmode").first()[0]

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 18, Finished, Available, Finished)

In [25]:
df_final = df_bronze.withColumn("ShipMode_ID", monotonically_increasing_id()+max_id+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 27, Finished, Available, Finished)

In [26]:
%%sql
SELECT * FROM ViewShipMode

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 28, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 2 fields>

In [32]:
%%sql
MERGE INTO Ecommere_Lh.gold_shipmode as gs
USING ViewShipMode as vs
ON gs.Ship_Mode = vs.Ship_Mode
WHEN MATCHED THEN
UPDATE set
Modified_TS = current_timestamp()

WHEN NOT MATCHED THEN
INSERT
(
    gs.ShipMode_id,
    gs.Ship_Mode,
    gs.Created_TS,
    gs.Modified_TS
)
VALUES
(
    vs.shipMode_ID,
    vs.Ship_Mode,
    CURRENT_TIMESTAMP(),
    CURRENT_TIMESTAMP()

)

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 34, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [33]:
%%sql
SELECT * FROM gold_shipmode

StatementMeta(, 8fb46c12-0525-4b48-a2ba-4679a0776c8d, 35, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>